## Import packages

In [1]:
import os
import h5py
import numpy as np
import pandas as pd
import tkinter
from tkinter.filedialog import askopenfilename, askopenfilenames
from collections import defaultdict
from nptdms import TdmsFile
print("done!")

done!


## Open an .hdf5 file and extract channels

In [ ]:
# This was Yu Lin's function:
def openHDF5file(in_path, channel_list = ['Channel A', 'Channel B', 'Output A', 'Output B'], curated_channel = None):
    hdf5_file = h5py.File(in_path, 'r')
     
    dataDict = defaultdict(list)
     
    for channel in hdf5_file.keys():
        
        #to fix hdf5 indexing
        if 'Channel' in channel or 'Output' in channel:
            trialKeysInHDF5 = list(hdf5_file[channel].keys()) # which has been sorted alphabatically as in strings. can be simulated by sorted([str(x) for x in range(numTrials)]) 
            sortI, numericalOrder = zip(*sorted(enumerate([int(x) for x in trialKeysInHDF5]),key=lambda i:i[1])) # resort numerically, and get the sorting indexes 
            
            if curated_channel is not None: 
                sortI, numericalOrder = zip(*sorted(enumerate([int(x) for x in list(hdf5_file[curated_channel].keys())]),
                                                    key=lambda i:i[1]))
                if 'Channel' in channel: trialKeysInHDF5 = list(hdf5_file[curated_channel].keys())
                if 'Output' in channel: sortI = [int(x)-1 for x in numericalOrder] #note, may not work as expected if very first trial is deleted ? but doesnt matter as output is usually the same across trials in a given file
#            print(channel, trialKeysInHDF5, sortI)
        else:
            trialKeysInHDF5 = list(hdf5_file[channel].keys())
            sortI = range(len(trialKeysInHDF5))
#            print('not sorted')
            
        for i in range(len(sortI)):
            correctedTrialKey = trialKeysInHDF5[sortI[i]] 
            dataDict[channel].append(np.array(hdf5_file[channel][correctedTrialKey]))
   
    extractedChannels = []
    
    for channel in channel_list:
        extractedChannels.append(dataDict[channel])

    
    if len(dataDict['Time']) >0:
        time = dataDict['Time'][0]
        dt = np.mean(np.diff(time))
    else:
        dt = 0.04
        time = np.linspace(0, len(dataDict['Channel A'][0])*dt, len(['Channel A'][0]))
    
    return extractedChannels, time, dt

### Load an .hdf5 file

In [2]:
# Choose a .hdf5 file to test the code
root = tkinter.Tk()
root.attributes('-topmost', True) # Make window appear on top
in_path = askopenfilename() # Open dialogue to select file
print(in_path)
root.destroy() # Close the root window

# Split the pathname path into a pair, (head, tail) where tail is the last pathname component and head is everything leading up to that.
folder_name = os.path.split(in_path)[0] # Get path until folder
file_name = os.path.split(in_path)[1] # Get filename
print(folder_name)
print(file_name)

D:/Dropbox (UCL - SWC)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells/dlpag_vgat_201201_c5_LDIBX_OP_VC_clear_nointerval_1.hdf5
D:/Dropbox (UCL - SWC)/Project_paginhibition/analysis/loose_seal/analysis_sample_cells
dlpag_vgat_201201_c5_LDIBX_OP_VC_clear_nointerval_1.hdf5


### Check how zip() and sorted() work

In [3]:
#print(in_path)
test_file = h5py.File(in_path, 'r')
print(list(test_file.keys()))
test_file['Channel B'].keys()

trialKeysInHDF5 = list(test_file['Channel B'].keys())
print(trialKeysInHDF5)

# This is how Yu Lin was doing the sorting:
dataDict = defaultdict(list)
sortI, numericalOrder = zip(*sorted(enumerate([int(x) for x in trialKeysInHDF5]), 
                                    key = lambda i:i[1]))

print(sortI)
print(numericalOrder)

['Channel A', 'Channel B', 'Output A', 'Output B', 'Sweeps_Analysis', 'Time']
['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '4', '5', '6', '7', '8', '9']
(0, 11, 22, 25, 26, 27, 28, 29, 30, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24)
(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31)


In [4]:
# How zip() and sorted() work:
a = ("John", "Charles", "Mike")
b = (2, 1, 3)
x = sorted(zip(b, a))
list(x)

[(1, 'Charles'), (2, 'John'), (3, 'Mike')]

### Test how the core of the function will work with one channel

In [16]:
test_file = h5py.File(in_path, 'r')

# Get trial keys from one of the channels:
a = list(test_file['Channel B'].keys())
print(a)

# Transform to integer:
b = [int(x) for x in a]
print(b)

# If trial keys don't start at 1, force them to (useful for "Output" channels):
c = [(x-(min(b))+1) for x in b]
print(c)

# Sort the integers:
d = sorted(c)
print(d)

# Use numpy argsort() to get the indices that would sort the array:
e = list(np.argsort(c, axis=None))
print(e)

# Turn integers into indices (i.e. starting from 0)
f = [(x-b[0]) for x in b]
print(f)

# Get a list of sorted trial keys
# Note this is not useful as you need the indices to access the keys in a sorted way so you can extract the data from the right sweep.
z = [x for _, x in sorted(zip(c, a))]
print(z)

corrected_test_file = defaultdict(list)
for i in range(len(e)):
    correctedTrialKey = a[e[i]]
    corrected_test_file['Channel B'].append(np.array(test_file['Channel B'][correctedTrialKey]))
print(corrected_test_file['Channel B'][0:10])

uncorrected_test_file = defaultdict(list)
for i in range(len(f)):
    uncorrectedTrialKey = a[f[i]]
    uncorrected_test_file['Channel B'].append(np.array(test_file['Channel B'][uncorrectedTrialKey]))
print(uncorrected_test_file['Channel B'][0:10])

['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '4', '5', '6', '7', '8', '9']
[1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 4, 5, 6, 7, 8, 9]
[1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 4, 5, 6, 7, 8, 9]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[0, 11, 22, 25, 26, 27, 28, 29, 30, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24]
[0, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 2, 29, 30, 3, 4, 5, 6, 7, 8]
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31']
[array([-15.15814815, -15.80213493, -17.41210188, ..., -36.0877179

In [6]:
#print(in_path)
test_file = h5py.File(in_path, 'r')
print(list(test_file.keys())) # Check channel names in file

trialKeysInHDF5 = list(test_file['Output B'].keys())
print(trialKeysInHDF5) # Check trial names in a channel

trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
print(trialKeysInHDF5_int)

trialKeysInHDF5_int.sort()
print(trialKeysInHDF5_int)

['Channel A', 'Channel B', 'Output A', 'Output B', 'Sweeps_Analysis', 'Time']
['4053', '4054', '4055', '4056', '4057', '4058', '4059', '4060', '4061', '4062', '4063', '4064', '4065', '4066', '4067', '4068', '4069', '4070', '4071', '4072', '4073', '4074', '4075', '4076', '4077', '4078', '4079', '4080', '4081', '4082', '4083']
[4053, 4054, 4055, 4056, 4057, 4058, 4059, 4060, 4061, 4062, 4063, 4064, 4065, 4066, 4067, 4068, 4069, 4070, 4071, 4072, 4073, 4074, 4075, 4076, 4077, 4078, 4079, 4080, 4081, 4082, 4083]
[4053, 4054, 4055, 4056, 4057, 4058, 4059, 4060, 4061, 4062, 4063, 4064, 4065, 4066, 4067, 4068, 4069, 4070, 4071, 4072, 4073, 4074, 4075, 4076, 4077, 4078, 4079, 4080, 4081, 4082, 4083]


In [7]:
print(range(len(list(test_file['Time'].keys())))) # check items in time
print(test_file['Output B'].keys()) # check trial names
test_file['Output B']['4053'] # check data in a sweep

range(0, 1)
<KeysViewHDF5 ['4053', '4054', '4055', '4056', '4057', '4058', '4059', '4060', '4061', '4062', '4063', '4064', '4065', '4066', '4067', '4068', '4069', '4070', '4071', '4072', '4073', '4074', '4075', '4076', '4077', '4078', '4079', '4080', '4081', '4082', '4083']>


<HDF5 dataset "4053": shape (1875,), type "<f8">

In [8]:
test_file = h5py.File(in_path, 'r')
a = list(test_file['Channel B'].keys())
a_curated = list(test_file['Sweeps_Analysis'].keys())
print(a)
print(a_curated)

b = [int(x) for x in a]
b_curated = [int(x) for x in a_curated]
print(b)
print(b_curated)

c = [(x-(min(b))+1) for x in b]
c_curated = [(x-(min(b_curated))+1) for x in b_curated]
print(c)
print(c_curated)

e = list(np.argsort(c))
e_curated = list(np.argsort(c_curated))
print(e)
print(e_curated)

f = list(np.argsort(b))
f_curated = list(np.argsort(b_curated))
print(f)
print(f_curated)


# Test data extraction using the corrected indices
dataDict = defaultdict(list)
extractedChannels = []
for i in range(len(f_curated)):
    corrected_a_curated = a_curated[f_curated[i]]
    dataDict['Sweeps_Analysis'].append(np.array(test_file['Sweeps_Analysis'][corrected_a_curated]))
print(list(dataDict['Sweeps_Analysis']))
extractedChannels.append(dataDict['Sweeps_Analysis'])
print(extractedChannels)

['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '3', '30', '31', '4', '5', '6', '7', '8', '9']
['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '7', '8', '9']
[1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 4, 5, 6, 7, 8, 9]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 7, 8, 9]
[1, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 2, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 3, 30, 31, 4, 5, 6, 7, 8, 9]
[4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 1, 2, 3]
[0, 11, 22, 25, 26, 27, 28, 29, 30, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24]
[15, 16, 17, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[0, 11, 22, 25, 26, 27, 28, 29, 30, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24]
[15, 16, 17, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13

In [11]:
test_file = h5py.File(in_path, 'r')

# Get keys from hdf5
trialKeysInHDF5 = list(test_file['Output B'].keys())
trialKeysInHDF5_curated = list(test_file['Sweeps_Analysis'].keys())
print(trialKeysInHDF5)
print(trialKeysInHDF5_curated)

# Convert to integers so you can sort numerically
trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
trialKeysInHDF5_curated_int = [int(x) for x in trialKeysInHDF5_curated]
print(trialKeysInHDF5_int)
print(trialKeysInHDF5_curated_int)

# Fix those cases when trial names start at a random number (e.g. sometimes "Output" start at "14197", whereas "Channel" always start at "1")
trialKeysInHDF5_int_from1 = [(x-min(trialKeysInHDF5_int)+1) for x in trialKeysInHDF5_int] # aka numerical order
print(trialKeysInHDF5_int_from1)

trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int_from1)) # aka sortIndices
print(trialKeysInHDF5_sorting_indices)

trialKeysInHDF5_sorting_indices_2 = list(np.argsort(trialKeysInHDF5_int)) # aka sortIndices
trialKeysInHDF5_curated_sorting_indices_2 = list(np.argsort(trialKeysInHDF5_curated_int)) # aka sortIndices
print(trialKeysInHDF5_sorting_indices_2)
print(trialKeysInHDF5_curated_sorting_indices_2)

trialKeysInHDF5_sorting_indices == trialKeysInHDF5_sorting_indices_2


# corrected_test_file = defaultdict(list)
# for i in range(len(trialKeysInHDF5_sorting_indices)):
#      correctedTrialKey = trialKeysInHDF5[trialKeysInHDF5_sorting_indices[i]]
#      corrected_test_file['Channel B'].append(np.array(test_file['Channel B'][correctedTrialKey]))

['4053', '4054', '4055', '4056', '4057', '4058', '4059', '4060', '4061', '4062', '4063', '4064', '4065', '4066', '4067', '4068', '4069', '4070', '4071', '4072', '4073', '4074', '4075', '4076', '4077', '4078', '4079', '4080', '4081', '4082', '4083']
['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '7', '8', '9']
[4053, 4054, 4055, 4056, 4057, 4058, 4059, 4060, 4061, 4062, 4063, 4064, 4065, 4066, 4067, 4068, 4069, 4070, 4071, 4072, 4073, 4074, 4075, 4076, 4077, 4078, 4079, 4080, 4081, 4082, 4083]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 7, 8, 9]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[15, 16, 17, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,

True

In [12]:
# For the curated version of an Output:
test_file = h5py.File(in_path, 'r')

# Get keys from hdf5
trialKeysInHDF5 = list(test_file['Output B'].keys())
trialKeysInHDF5_curated = list(test_file['Sweeps_Analysis'].keys())
print(trialKeysInHDF5)
print(trialKeysInHDF5_curated)

# Convert to integers so you can sort numerically
trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
trialKeysInHDF5_curated_int = [int(x) for x in trialKeysInHDF5_curated]
print(trialKeysInHDF5_int)
print(trialKeysInHDF5_curated_int)

# Fix those cases when trial names start at a random number (e.g. sometimes "Output" start at "14197", whereas "Channel" always start at "1")
trialKeysInHDF5_int_from1 = [(x-min(trialKeysInHDF5_int)+1) for x in trialKeysInHDF5_int] # aka numerical order
print(trialKeysInHDF5_int_from1)

# trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int_from1)) # aka sortIndices
# print(trialKeysInHDF5_sorting_indices)

trialKeysInHDF5_sorting_indices_2 = list(np.argsort(trialKeysInHDF5_int)) # aka sortIndices
trialKeysInHDF5_curated_sorting_indices_2 = list(np.argsort(trialKeysInHDF5_curated_int)) # aka sortIndices
print(trialKeysInHDF5_sorting_indices_2)
print(trialKeysInHDF5_curated_sorting_indices_2)

a = sorted(trialKeysInHDF5_curated_int)
b = trialKeysInHDF5_int_from1
print(a)
print(b)
print(b.index(a[0]))
c = [b.index(a[i]) for i in range(len(a))]
print(c)
test_file['Output B'][trialKeysInHDF5[c[0]]]

['4053', '4054', '4055', '4056', '4057', '4058', '4059', '4060', '4061', '4062', '4063', '4064', '4065', '4066', '4067', '4068', '4069', '4070', '4071', '4072', '4073', '4074', '4075', '4076', '4077', '4078', '4079', '4080', '4081', '4082', '4083']
['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '7', '8', '9']
[4053, 4054, 4055, 4056, 4057, 4058, 4059, 4060, 4061, 4062, 4063, 4064, 4065, 4066, 4067, 4068, 4069, 4070, 4071, 4072, 4073, 4074, 4075, 4076, 4077, 4078, 4079, 4080, 4081, 4082, 4083]
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 7, 8, 9]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[15, 16, 17, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[1, 2, 3, 4, 5, 6, 7, 8,

<HDF5 dataset "4059": shape (1875,), type "<f8">

In [29]:
# Check the output channel loop
trialKeysInHDF5 = list(test_file['Output B'].keys())
print(trialKeysInHDF5)
trialKeysInHDF5_int = [int(x) for x in trialKeysInHDF5]
print(trialKeysInHDF5_int)
trialKeysInHDF5_sorting_indices = list(np.argsort(trialKeysInHDF5_int))
print(trialKeysInHDF5_sorting_indices)
# Transform them so they start from 1 and can be compared to the curated_channel keys:
trialKeysInHDF5_int_from1 = [(x-min(trialKeysInHDF5_int)+1) for x in trialKeysInHDF5_int]
print(trialKeysInHDF5_int_from1)

trialKeysInHDF5_curated = list(test_file['Sweeps_Analysis'].keys())
print(trialKeysInHDF5_curated)
trialKeysInHDF5_curated_int = [int(x) for x in trialKeysInHDF5_curated]
print(trialKeysInHDF5_curated_int)
trialKeysInHDF5_curated_sorting_indices = list(np.argsort(trialKeysInHDF5_curated_int))
print(trialKeysInHDF5_curated_sorting_indices)

# Sort the curated integer keys so you can use them in the list.index() step
trialKeysInHDF5_curated_int_sorted = sorted(trialKeysInHDF5_curated_int)
print(trialKeysInHDF5_curated_int_sorted)

# For the sorted curated trial keys, find the index of the value matching each curated_channel trial.
# Use this as the sorting indices
trialKeysInHDF5_sorting_indices = [trialKeysInHDF5_int_from1.index(trialKeysInHDF5_curated_int_sorted[i]) for i in range(len(trialKeysInHDF5_curated_int_sorted))]
print(trialKeysInHDF5_sorting_indices)

['4053', '4054', '4055', '4056', '4057', '4058', '4059', '4060', '4061', '4062', '4063', '4064', '4065', '4066', '4067', '4068', '4069', '4070', '4071', '4072', '4073', '4074', '4075', '4076', '4077', '4078', '4079', '4080', '4081', '4082', '4083']
[4053, 4054, 4055, 4056, 4057, 4058, 4059, 4060, 4061, 4062, 4063, 4064, 4065, 4066, 4067, 4068, 4069, 4070, 4071, 4072, 4073, 4074, 4075, 4076, 4077, 4078, 4079, 4080, 4081, 4082, 4083]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30]
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '7', '8', '9']
[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 7, 8, 9]
[15, 16, 17, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[6, 7, 8, 9, 10, 11, 12,

### Check data extraction

In [47]:
dataDict['Output B']

[array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 0., 0., 0.]),
 array([0., 0., 0., ..., 

In [55]:
# Define empty dictionary to populate with correctly sorted data:
data_dict = defaultdict(list)
# Define empty dictionary to populate with corrected trialKeys:
key_dict = defaultdict(list)
print(len(trialKeysInHDF5_sorting_indices))
for i in range(len(trialKeysInHDF5_sorting_indices)):
    correctedTrialKey = trialKeysInHDF5[trialKeysInHDF5_sorting_indices[i]] 
    data_dict['Output B'].append(np.array(test_file['Output B'][correctedTrialKey]))
    key_dict['Output B'].append(correctedTrialKey)
    print(data_dict['Output B'])
    print(key_dict['Output B'])

print(len(trialKeysInHDF5_curated))
print(len(key_dict['Output B']))
extracted_channels = []
corrected_trial_keys = []

# Keep only the useful channels and their trial keys:
extracted_channels.append(data_dict['Output B'])
print(len(extracted_channels[0]))
corrected_trial_keys.append(key_dict['Output B'])
print(len(corrected_trial_keys[0]))
    
# Create data frame of data:
channel_names = ['Output B']
channels_data_frame = pd.DataFrame(extracted_channels, index = channel_names, columns = corrected_trial_keys[0])
channels_data_frame

18
[array([0., 0., 0., ..., 0., 0., 0.])]
['4059']
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]
['4059', '4060']
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]
['4059', '4060', '4061']
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]
['4059', '4060', '4061', '4062']
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]
['4059', '4060', '4061', '4062', '4063']
[array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.]), array([0., 0., 0., ..., 0., 0., 0.])]
['4059', '4060', '4061', '4062', '4063', '4064']
[array([0., 0., 0.,

,4059,4060,4061,4062,4063,4064,4065,4066,4067,4068,4069,4070,4071,4072,4073,4074,4075,4076
Output B,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


### How to make a pandas data frame

In [13]:
# np.array(test_file['Sweeps_Analysis']['13'])
# channel_list = ['Channel A', 'Channel B']
# data = [np.arange(0, 18), np.arange(100, 118)]
# print(data)
# print(extractedChannels)
indices = ['Sweeps_Analysis']
df = pd.DataFrame(extractedChannels, index = indices, columns = a_curated)
df

,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,7,8,9
Sweeps_Analysis,"[-11.616220826606902, -7.752300066135383, -8.7...","[9.635343909020817, 5.771422947326991, 11.8892...","[0.29753499493087243, -0.6684452226398963, 1.9...","[-20.632035761040658, -17.734095272842332, -22...","[-11.938214221299425, -13.870174582953112, -14...","[-7.752300066135383, -8.718280260436195, -9.68...","[-4.532366064980151, -0.6684452226398963, -5.8...","[5.449429535875142, 6.737403183549116, -0.3464...","[-3.2443924558328088, -4.210372663158741, -0.9...","[8.669363664395322, 4.805442713904577, 0.61952...","[-24.817949755377462, -25.13994313739636, -29....","[-10.97223403629284, -17.734095272842332, -15....","[-10.00625384849881, -8.074293464545597, -10.0...","[6.093416359089918, 6.737403183549116, 6.09341...","[-2.922399052770176, -0.3464518170935623, -2.2...","[-12.58220100975559, -10.650240640671248, -12....","[-12.260207615682315, -11.294227431604716, -14...","[-3.2443924558328088, 0.619528401409027, -1.95..."
